### 매출에 영향력 높은 후행 지표 찾기

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
base_path = "C:/Users/dukkubi"
df = pd.read_csv(base_path + "영업팀 매출 데이터.csv")
form = pd.read_csv(base_path + "피터팬의 좋은방구하기 입점문의 접수(응답).csv")
agent = pd.read_csv(base_path + "중개사 데이터.csv")
first_party_db = pd.read_csv(base_path + "first_party_db.csv")

영업팀 매출 데이터에 중개사를 식별하는 key가 존재하지 않는다.<br>
따라서, 지역 + 상호 + 대표자 데이터를 공공 데이터의 지역 + 상호 + 대표자와 join 하여 등록번호를 부여했다.

In [ ]:
df['지역 상호 대표자'] = df['지역']  + ' ' + df['상호'] + ' ' + df['대표자']

first_party_db['지역 상호 대표자'] = df['지역']  + ' ' + df['상호'] + ' ' + df['대표자']
first_party_db = first_party_db[['지역 상호 대표자', 'resgister_number']]

df = df.merge(first_party_db, on='지역 상호 대표자', how='left')

#### 전처리

In [ ]:
# '매출액' 열의 빈 문자열을 0으로 대체하고 '-' 기호 제거하고 쉼표(,) 제거한 후에 숫자형으로 변환
df['매출액'] = df['매출액'].replace('', '0').str.replace(',', '').astype(int)

#### 신규가입 중개사 vs 재계약(연장) 중개사 비교

In [ ]:
# 신규 가입 중개사 수와 재개약(연장) 중개사 수 비교 
print(df.groupby('신규연장', dropna=False).size().reset_index(name='count'))

# 신규 가입 중개사의 매출과 연장 중개사의 매출 비교
print(df.groupby('신규연장', dropna=False)['매출액'].sum().reset_index())

연장 중개사 수, 매출 비율이 더 높은 점을 확인했다.<br>
평균적으로 중개사가 몇 개월 단위로, 몇 회 연장을 하는지도 확인을 해야한다.<br>
<br>
ex) 1개월 당 꾸준히 100명의 신규 중개사 유입
- 평균 연장 횟수: 3회
- 1회 연장 당 평균적으로 걸리는 기간: 3개월<br>

위 조건에서 작년 이번 달 처음 100명의 신규 중개사가 유입되었다고 가정하면 다음과 같은 결과가 나온다.

- 1개월 뒤 : 신규 중개사 100명, 연장 중개사 0명
- 2개월 뒤 : 신규 중개사 100명, 연장 중개사 0명
- 3개월 뒤 : 신규 중개사 100명, 연장 중개사 100명
- 4개월 뒤 : 신규 중개사 100명, 연장 중개사 100명
- 5개월 뒤 : 신규 중개사 100명, 연장 중개사 100명
- 6개월 뒤 : 신규 중개사 100명, 연장 중개사 200명
- 7개월 뒤 : 신규 중개사 100명, 연장 중개사 200명
- 8개월 뒤 : 신규 중개사 100명, 연장 중개사 200명
- 9개월 뒤 : 신규 중개사 100명, 연장 중개사 300명
- 10개월 뒤 : 신규 중개사 100명, 연장 중개사 300명
- 11개월 뒤 : 신규 중개사 100명, 연장 중개사 300명
- 12개월 뒤 : 신규 중개사 100명, 연장 중개사 300명

#### 평균 연장 횟수 구하기

In [ ]:
from datetime import date
from dateutil.relativedelta import relativedelta

## 평균 연장 횟수

# 중개사별 첫 구매 날짜
first_purchase_date = df.loc[df['신규연장'] == '신규', ['날짜', 'register_number']].rename(columns={'날짜': '신규가입날짜'})

# 월 단위로 변경
first_purchase_date['신규가입날짜'] =  pd.to_datetime(first_purchase_date['신규가입날짜']).dt.strftime('%Y-%m')

# 기존 데이터에 첫 구매 날짜 join
df = df.merge(first_purchase_date, on='register_number', how='left')

# 연장 날짜 - 첫 구매 날짜 계산
df['연장경과개월'] = (
    (pd.to_datetime(df['날짜']).dt.year - pd.to_datetime(df['신규가입날짜'], format='%Y-%m').dt.year) * 12 +
    (pd.to_datetime(df['날짜']).dt.month - pd.to_datetime(df['신규가입날짜'], format='%Y-%m').dt.month)
)

# 첫 구매일로부터 1개월 단위로 연장한 횟수
df_cohort = df.groupby(['신규가입날짜', '연장경과개월'])['register_number'].nunique().reset_index()

# 11개월 경과까지의 데이터만 선택. 12개월치 데이터가 없는 중개사는 선택 X
year_month_of_12_ago = (date.today() - relativedelta(months=12)).strftime('%Y-%m')
condition1 = (df_cohort['연장경과개월'] < 12)
condition2 = (df_cohort['신규가입날짜'] <= year_month_of_12_ago)
condition3 = (df_cohort['연장경과개월'] != 0)
df_cohort = df_cohort.loc[condition1 & condition2 & condition3]

# 평균 연장 횟수
년월기준신규중개사수 = df_cohort.loc[df_cohort['연장경과개월'] == 1].groupby('신규가입날짜')['register_number'].sum().reset_index()['register_number'].sum()
년월기준연장중개사수 = df_cohort.loc[df_cohort['연장경과개월'] != 1].groupby('신규가입날짜')['register_number'].sum().reset_index()['register_number'].sum()
print(년월기준연장중개사수/년월기준신규중개사수)

#### 1회 연장 당 평균적으로 걸리는 기간 구하기

#### 신규 가입 후, 3개월 안에 이탈하는 이탈률 구하기

#### 1차 후행 지표 선정

1회 연장 당 평균적으로 걸리는 기간은 업계 특성상 쉽게 바뀔 수 없다.<br>
억지로 기간을 줄이려고 하면 역효과가 나타날 수 있다.<br>
또한, 평균 연장 횟수가 같이 늘어나지 않는 이상 연장 기간이 줄어드는건 수치상 큰 의미는 없다.<br>
따라서, 매출을 늘리기 위해 개선이 필요한 지표는 다음과 같다.

**개선이 필요한 지표**
- 신규 중개사 수
- 평균 연장 횟수
- ARPPU

이를 식으로 대략적으로 나타내면 다음과 같다.

$ 매출 = (신규 중개사 수 + (신규 중개사 수 \times 평균 연장 횟수)) * ARPPU $

신규 중개사 수는 단위가 크지만, 평균 연장 횟수는 단위가 작으므로 1회만 늘려도 큰 효과를 볼 수 있다.<br>
ex) 1개월당 꾸준히 유입되는 신규 중개사가 50명이 넘는 상황에서는 신규 중개사 10명 늘리는 것보다 평균 연장 횟수 1회 늘리는 게 더 큰 효과를 본다.<br>

ARPPU는 영업팀, 경영전략팀에서 전담해서 맡고 있기 때문에 ARPPU에 악영향을 주지 않는 방향으로 나머지 2지표에 집중하기로 했다.

- 평균 연장 횟수는 서비스 팀, 마케팅 팀에서 서포트 가능.
- 신규 중개사 수는 마케팅 팀에서 서포트 가능.

평균 연장 횟수에 기여하는 지표가 무엇인지 조사해보았다.

지금까지는 평균 연장 횟수를 전체 중개사 대상으로 조사했지만, 개별 조사도 필요하다.<br>
평균 연장 횟수의 분산을 통해 상위 20% 그룹, 중간 그룹, 하위 20% 그룹으로 나누었다.

각 그룹 대상으로 다시 지표를 살펴보았다.

#### 유입 채널에 따른 유입수, 전환율 차이